In [ ]:
pip install mediapipe opencv-python

In [ ]:
import cv2
import mediapipe as mp

# MediaPipe Hand Detection Setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

def classify_gesture(landmarks):
    thumb_tip = landmarks[4]
    index_tip = landmarks[8]
    middle_tip = landmarks[12]
    ring_tip = landmarks[16]
    pinky_tip = landmarks[20]
    wrist = landmarks[0]

    # Distance between thumb and index
    thumb_index_dist = abs(thumb_tip.x - index_tip.x)

    if all(tip.y < wrist.y for tip in [index_tip, middle_tip, ring_tip, pinky_tip]) and thumb_tip.x < wrist.x:
        return "Open Palm"
    elif thumb_index_dist < 0.05 and middle_tip.y > wrist.y:
        return "OK Sign"
    elif all(lm.y < wrist.y for lm in [index_tip, middle_tip]):
        return "Victory"
    elif all(lm.y > wrist.y for lm in [index_tip, middle_tip]):
        return "Fist"
    else:
        return "Unknown"


# Open Webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(rgb)
    gesture = "No Hand"

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            gesture = classify_gesture(hand_landmarks.landmark)

    # Show gesture on screen
    cv2.putText(frame, f"Gesture: {gesture}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3)
    cv2.imshow("Hand Gesture Recognition", frame)

    # Quit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
